# OLIMPIJSKE IGRE

Analiza dosežkov na olimpijskih igrah po državah in pogostosti posameznih športnih disciplin na olimpijskih igrah.

## Uvoz podatkov

In [76]:
import pandas as pd            # za delo s tabelami
import seaborn as sns          # za lepo risanje grafov             # da se grafi prikažejo v notebooku
%matplotlib inline

## ker bomo delali z velikimi razpredelnicami, povemo, da naj se vedno izpiše le 10 vrstic
pd.set_option("display.max_rows", 10)
# pd.options.display.max_rows = 10      # druga opcija, oboje je ok

# naložimo razpredelnico, s katero bomo delali
MEDALJE = pd.read_csv("oi8.3_medalje_pregled.csv", index_col=0)
DISCIPLINE = pd.read_csv("oi9.3_discipline_pregled.csv", index_col=0)

In [77]:
# MEDALJE
# DISCIPLINE

In [78]:
MEDALJE_SUMMER = MEDALJE.filter(like="Summer").astype(int).rename(columns=lambda x: x.replace("-Summer", "")).loc[lambda df: ~(df == 0).all(axis=1)]
MEDALJE_WINTER = MEDALJE.filter(like="Winter").astype(int).rename(columns=lambda x: x.replace("-Winter", "")).loc[lambda df: ~(df == 0).all(axis=1)]

In [79]:
DISCIPLINE_SUMMER = DISCIPLINE.filter(like="Summer").astype(int).rename(columns=lambda x: x.replace("-Summer", "")).loc[lambda df: ~(df == 0).all(axis=1)]
DISCIPLINE_WINTER = DISCIPLINE.filter(like="Winter").astype(int).rename(columns=lambda x: x.replace("-Winter", "")).loc[lambda df: ~(df == 0).all(axis=1)]

##

## Splošno

Število olimpijskih iger

In [80]:
print(f"Skupaj olimpijskih iger: {len(DISCIPLINE.columns)}")
print(f"Poletne olimpijske igre: {len(DISCIPLINE_SUMMER.columns)}")
print(f"Zimske olimpijske igre: {len(DISCIPLINE_WINTER.columns)}")

Skupaj olimpijskih iger: 54
Poletne olimpijske igre: 30
Zimske olimpijske igre: 24


Lokacija olimpijskih iger

In [81]:
kraji = DISCIPLINE.columns.str.split("-", n=2).str[1].value_counts()
kraji_najvec_oi = kraji[kraji == kraji.max()]

print(f"Največrat so se Olimpijske igre odvijale v {", ".join(kraji_najvec_oi.index)} in sicer {kraji.max()}-krat.")     #sklanjanje!!

Največrat so se Olimpijske igre odvijale v Paris, London in sicer 3-krat.


In [82]:
tokyo_leta = DISCIPLINE.columns[DISCIPLINE.columns.str.contains("-Tokyo-")].str.split("-", n=2).str[0]
print(f"V Tokyu so bile Olimpijske igre {kraji['Tokyo']}-krat, in sicer leta {', '.join(tokyo_leta)}.")

V Tokyu so bile Olimpijske igre 2-krat, in sicer leta 1964, 2020.


In [83]:
mesto = "Tokyo"     # mesto za katero me zanima

if mesto not in kraji:
    print(f'Mesto {mesto} ni nikoli gostilo Olimpijskih iger.')
else:
    katera_leta = DISCIPLINE.columns[DISCIPLINE.columns.str.contains(f"-{mesto}-")].str.split("-", n=2).str[0]
    print(f"Mesto {mesto} je gostilo Olimpijske igre {kraji[mesto]}-krat, in sicer leta {', '.join(katera_leta)}.")

Mesto Tokyo je gostilo Olimpijske igre 2-krat, in sicer leta 1964, 2020.


In [84]:
mesto = "Ljubljana"     # mesto za katero me zanima

if mesto not in kraji:
    print(f'Mesto {mesto} ni nikoli gostilo Olimpijskih iger.')
else:
    katera_leta = DISCIPLINE.columns[DISCIPLINE.columns.str.contains(f"-{mesto}-")].str.split("-", n=2).str[0]
    print(f"Mesto {mesto} je gostilo Olimpijske igre {kraji[mesto]}-krat, in sicer leta {', '.join(katera_leta)}.")

Mesto Ljubljana ni nikoli gostilo Olimpijskih iger.


## random

Koliko različnih držav je dobilo medalje na olimpijskih igrah

In [48]:
print(len(MEDALJE))

157


#### Dobljene medalje na olimpijskih igrah

In [49]:
drzava = MEDALJE.index.to_frame(index=False)
stolpci_skupaj = MEDALJE.loc[:, MEDALJE.columns.str.contains("skupaj", case=False)]

MEDALJE_SKUPAJ = pd.concat([drzava, stolpci_skupaj], axis=1)
MEDALJE_SKUPAJ.columns = MEDALJE_SKUPAJ.columns.str.replace("-Skupaj", "", case=False)

print(MEDALJE_SKUPAJ)

                         Država  1896-Athens-Summer  1900-Paris-Summer  \
0                           AIN                 NaN                NaN   
1                   Afghanistan                 NaN                NaN   
2                       Albania                 NaN                NaN   
3                       Algeria                 NaN                NaN   
4                     Argentina                 NaN                NaN   
...                         ...                 ...                ...   
Vietnam                     NaN                 0.0                0.0   
Virgin Islands, US          NaN                 0.0                0.0   
Yugoslavia                  NaN                 0.0                0.0   
Zambia                      NaN                 0.0                0.0   
Zimbabwe                    NaN                 0.0                0.0   

                    1904-St_Louis-Summer  1908-London-Summer  \
0                                    NaN       

Koliko medalj se je podelilo vsako leto

## Medalje